<a href="https://colab.research.google.com/github/AI-Sunmoon2021/ImageProcessingProjects/blob/main/vgg16_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#モジュールオブジェクトのインポート
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Dropout, Flatten, Input
from keras.applications.vgg16 import VGG16
from keras.models import Model, Sequential
from keras import optimizers
from os import listdir


In [8]:

#イメージサイズの定義をする
image_size = 50

#listdir('./xx/')ファイルの中にある画像を入れる。
path_bh = [filename for filename in listdir('/content/meats/rare/') if not filename.startswith('.')]
path_bu = [filename for filename in listdir('/content/meats/welldone/') if not filename.startswith('.')]
path_rollar = [filename for filename in listdir('/content/meats/test/') if not filename.startswith('.')]



#各ファイルを順番に呼び出す。（ファイルにある個数分繰り返す。）
#OpenCVで画像サイズをリサイズして統一する。
#img_にリサイズしたものを加える。

img_bh = []
img_bu = []
img_rollar = []
for i in range(len(path_bh)):
   img = cv2.imread('/content/meats/rare/'+ path_bh[i])
   img = cv2.resize(img,(image_size,image_size))
   img_bh.append(img)
for i in range(len(path_bu)):
   img = cv2.imread('/content/meats/welldone/'+ path_bu[i])
   img = cv2.resize(img,(image_size,image_size))
   img_bu.append(img)
for i in range(len(path_rollar)):
   img = cv2.imread('/content/meats/test/'+ path_rollar[i])
   img = cv2.resize(img,(image_size,image_size))
   img_rollar.append(img)



X = np.array(img_bh + img_bu + img_rollar) #行列をくっつける。
y = np.array([0]*len(img_bh) + [1]*len(img_bu) + [2]*len(img_rollar)) #０、１、２と種類ごとに番号をつける。
rand_index = np.random.permutation(np.arange(len(X))) #np.arange(x)の配列をランダムに並べ替えてrand_indexに入れる。
X = X[rand_index] #Xをrand_index順に並び替える
y = y[rand_index] #yをrand_index順に並び替える

#データを分割する。
X_train = X[:int(len(X)*0.8)] #トレーニングデータ８割未満
y_train = y[:int(len(y)*0.8)]
X_test = X[int(len(X)*0.8):] #トレーニングデータ以外
y_test = y[int(len(y)*0.8):]

#各属性を表示する。
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

y_train = to_categorical(y_train) #One-Hotベクトルを作成する。
y_test = to_categorical(y_test) #One-Hotベクトルを作成する。

# 転移学習（VGG16モデル)を活用する。ImageNetで事前学習した重みを読み込む。
input_tensor = Input(shape=(image_size, image_size, 3)) 
vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

#特徴抽出部分以降に新しく他の層を追加するにために、あらかじめVGGとは別のモデル（ここではtop_model）を定義する。
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16.output_shape[1:])) #VGGを結合する。

top_model.add(Dense(256, activation="sigmoid")) #1つ目の全結合層の出力ユニット数は256,活性化関数はshigmoid.
top_model.add(Dropout(0.5))                     #過学習を防ぐ。
top_model.add(Dense(64, activation='sigmoid'))
top_model.add(Dropout(0.5))
top_model.add(Dense(32, activation='sigmoid'))
top_model.add(Dropout(0.5))
top_model.add(Dense(3, activation='softmax')) #ソフトマックス関数を用いて3次元で出力する。

## vgg16とtop_modelを連結してください
model = Model(inputs=vgg16.input, outputs=top_model(vgg16.output))

#15番目の層までは固定し、それ以降のものを繰り返し学習する。
for layer in model.layers[:15]:
   layer.trainable = False

#コンパイルする。
model.compile(loss='categorical_crossentropy', #多クラス分類の損失関数を定義する。
             optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), #最適化関数を定義する。
             metrics=['accuracy']) #評価関数を定義する。

#X_train, y_trainを用いて学習を行う。テストデータは、X_test, y_test。
history = model.fit(X_train, y_train, batch_size=32, epochs=25, verbose=1, validation_data=(X_test, y_test))

#精度の評価する。
score = model.evaluate(X_test, y_test, batch_size=32, verbose=0)
print('validation loss:{0[0]}\nvalidation accuracy:{0[1]}'.format(score))

#モデルを保存する。
model.save("my_model.h5")

(376, 50, 50, 3)
(376,)
(94, 50, 50, 3)
(94,)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/25
12/12 [==============================] - 21s 2s/step - loss: 1.2744 - accuracy: 0.3401 - val_loss: 1.0803 - val_accuracy: 0.4362
Epoch 2/25
12/12 [==============================] - 17s 1s/step - loss: 1.2616 - accuracy: 0.3331 - val_loss: 1.0723 - val_accuracy: 0.4362
Epoch 3/25
12/12 [==============================] - 18s 1s/step - loss: 1.2000 - accuracy: 0.3812 - val_loss: 1.0637 - val_accuracy: 0.4362
Epoch 4/25
12/12 [==============================] - 18s 1s/step - loss: 1.1238 - accuracy: 0.4172 - val_loss: 1.0556 - val_accuracy: 0.4362
Epoch 5/25
12/12 [==============================] - 18s 1s/step - loss: 1.2489 - accuracy: 0.3835 - val_loss: 1.0482 - val_accuracy: 0.4362
Epoch 6/25
12/12 [==============================] - 18s 1s/step - loss: 1.1554 - accuracy: 0.4380 - val_loss: 1.0413 - val_accuracy: 0.4362
Epoch 7/25
12/12 [==============================] - 18s 1s/step - loss: 1.2314 - accuracy: 0.3336 - val_loss: 1.0347 - val_accuracy: 0.4362
Epoch 8/25
12/12 [==